# Fault-Tolerant Quantum Error Correction using the Steane Error Correction Scheme

## 1. Introduction

Quantum computing is susceptible to environmental interference, thus quantum error correction and fault-tolerant techniques are proposed for practical quantum computation.

Quantum error correction uses an error-correcting code (such as Steane Code) to protect computational qubits, which encodes the computational qubits into a bundle of qubits (called logical qubits or *codewords*) and preforms an error correction procedure in case of errors.

The error correction procedure, in the theory of quantum error correction, is assumed to carry out perfectly, i.e., no more errors will happen during error correction. Obviously, it is an unrealistic assumption. To put the theory into practice, fault-tolerant techniques are proposed and applied.

In this tutorial, we discuss a fault-tolerant error correction scheme, the Steane Error Correction scheme, and verify its fault-tolerant property by applying it to error correction in a faulty simulated environment. We use Steane Code as the error-correcting code in this tutorial, but the Steane Error Correction scheme can work on any Calderbank-Shor-Steane (CSS) codes.

We first summarize some background knowledge about Steane Code and present a theoretical implementation of its error correcting procedure according to the theory of quantum error correction. Then we explain the error propagation problem in the theoretical implementation to reveal why fault tolerance is necessary. We next present the definition and the requirements of fault-tolerant quantum error correction, elaborate the Steane Error Correction scheme along with its Q# implementation and finally verify that the scheme satisfies the fault-tolerant requirements.

## 2. Backgroud : Steane Code

Steane Code is a $[[ 7,1,3 ]]$ stabilizer code and also a CSS code. It means that Steane Code uses $7$ qubits to encode $1$ computational qubit and the minimal distance $d$ between two valid codewords is $3$. Steane Code can correct single qubit errors ($t = \lfloor (d - 1) / 2 \rfloor = 1$). The parity check matrix of Steane Code is 

$$
\begin{bmatrix} 1 & 0 & 1 & 0 & 1 & 0 & 1 \\ 0 & 1 & 1 & 0 & 0 & 1 & 1 \\ 0 & 0 & 0 & 1 & 1 & 1 & 1 \end{bmatrix}
$$

For classical linear codes, the parity check matrix is used to detect the type of errors by regarding the codewords as column vectors and multiplying the parity check matrix and the vector. The procedure of detection is called *syndrome computation*, and the result of computation is a bit-string called *syndrome*. We then correct errors according to the syndrome via a mapping from syndromes to error correcting operations. We refer to the whole procedure including detecting and correcting as *error correction*.

<div>
    <img src="img/error-correction.png" width="50%" style="width: 50%; margin: 15px auto 15px auto;"/>
</div>

For quantum stabilizer codes, replacing $1$ in the parity check matrix with $X$($Z$) and $0$ with $I$, we will get its stabilizers. The stabilizers of Steane Code are:

$$
\begin{align}
S_1 = X \otimes I \otimes X \otimes I \otimes X \otimes I \otimes X \\ 
S_2 = I \otimes X \otimes X \otimes I \otimes I \otimes X \otimes X \\ 
S_3 = I \otimes I \otimes I \otimes X \otimes X \otimes X \otimes X \\ \\
S_4 = Z \otimes I \otimes Z \otimes I \otimes Z \otimes I \otimes Z \\ 
S_5 = I \otimes Z \otimes Z \otimes I \otimes I \otimes Z \otimes Z \\ 
S_6 = I \otimes I \otimes I \otimes Z \otimes Z \otimes Z \otimes Z
\end{align}
$$

A valid codeword is a +1-eigenstate of all the stabilizers. Measuring the codeword using these stabilizers can detect the type of errors (syndrome computation). For Steane Code, the result of measurement is a 6 bit-string (syndrome). The 
error correcting operations are of form

$$ P_1 \otimes P_2 \otimes ... \otimes P_n $$

where $P_i$ is one of Pauli operations ($X, Y, Z$) or identity ($I$).

Usually, we need a lookup table to enumerate the mapping from syndromes to error correcting operations. For Steane Code, there are $2^6 - 1 = 63$ types of errors. (All 0 syndrome means no error.) But for CSS codes, we have another view of the errors.

The $Z$-generators ($S_{4,5,6}$) detects bit flip errors. Measuring these $Z$-generators yields a 3 bit-string for $2^3 - 1 = 7$ types of errors, which corresponds to a bit flip of a certain qubit of the 7-qubit codewords. Similarly, the $X$-generators detects phase flip errors. Therefore, for CSS codes, we can separate the whole error correction into two, one for correcting bit flip errors, and another for phase flip errors.

Before we talk more about the error correction procedure using Steane Code, let us first learn about the basic usage of Steane Code in Q#.

### Example: Steane Code in Q# #

Q# Error Correction Library encapsulates several routines about encoding, decoding, syndrome computation and error correcting with Steane Code. We discuss encoding, decoding and syndrome computation routines in this example, and we left error correcting in the following sections.

#### Encoding and Decoding

We first construct a non-trivial state to begin with, and encode it with Steane Code. Then we perform decode and check its state. The following code shows the procedure.

In [1]:
open Microsoft.Quantum.Math; // for PI
open Microsoft.Quantum.ErrorCorrection; // import Error Correction Library

operation EncodeAndDecode() : Unit {
    // Steance Code uses 7 qubits to encode a state, we allocate one qubit `data` to represent the unencoded state,
    // and 6 auxiliary qubits to compose a codeword
    using ((data, auxQubits) = (Qubit(), Qubit[6])) {
        Rx(PI() / 3.0, data); // Constuct a non-trivial state to begin with
        
        let code = SteaneCode(); // SteaneCode encapsulates all necessary routines.
        let (encode, decode, syndMeasX, syndMeasZ) = code!; // Unwrap routines, we will use `encode` and `decode` here
        
        let codeword = encode!([data], auxQubits); // Encode data
        
        let (decodedData, decodedAux) = decode!(codeword); // Decode the codeword
        
        // Check the state
        Adjoint Rx(PI() / 3.0, data); // Undo rotation
        Assert([PauliZ], [data], Zero, "State doesn't return to |0>"); // The state should be |0>
    }
}

EncodeAndDecode

In [2]:
%simulate EncodeAndDecode

()

Internally, the encoding operation consists of a sequence of basic unitary operations. `SteaneCodeEncoderImpl` gives the implementation. Decoding operation is the adjoint operation of encoding.

In [ ]:
// https://github.com/microsoft/quantumlibraries/blob/master/Standard/src/ErrorCorrection/7QubitCode.qs

operation SteaneCodeEncoderImpl (data : Qubit[], scratch : Qubit[]) : Unit {
    body (...) {
        H(scratch[0]);
        H(scratch[2]);
        H(scratch[5]);
        CNOT(data[0], scratch[4]);
        CNOT(scratch[5], scratch[1]); // 1
        CNOT(scratch[5], scratch[3]); // 2
        CNOT(scratch[1], data[0]);
        CNOT(scratch[2], scratch[4]);
        CNOT(scratch[0], scratch[4]);
        CNOT(scratch[4], scratch[5]); // 3
        CNOT(scratch[2], scratch[3]);
        CNOT(scratch[0], scratch[1]);
    }

    adjoint invert;
}

As we can see, encoding involves many two qubit operations (CNOT), and one qubit occurs in many CNOT operations. For example, `scratch[5]` are used 3 times (marked by `1,2,3`). Thus, if a single qubit error takes place during encoding (e.g. an error on `scratch[5]`), it may propagate to many other qubits (e.g. `scratch[1, 3, 4]`), causing multiple qubit errors which cannot be corrected by Steane Code. We will review this point later.

### Syndrome Computation

As we discuss before, syndrome computation in Steane Code is performed by measuring the stabilizers. When we unwrap `SteaneCode`, we also get `syndMeasX` and `syndMeasZ`:

```
let (encode, decode, syndMeasX, syndMeasZ) = code!;
```

From the definition of `SteaneCode` below, we can confirm that the syndromes are actually computed by measurement where `xg` and `zg` corresponds to $S_{1-3}$ and $S_{4-6}$ respectively.

In [ ]:
// https://github.com/microsoft/quantumlibraries/blob/master/Standard/src/ErrorCorrection/7QubitCode.qs

function SteaneCode () : CSS {
    let e = EncodeOp(EncodeIntoSteaneCode);
    let d = DecodeOp(DecodeFromSteaneCode);
    // S1-3
    let xg = 
        [ 
            [PauliX, PauliI, PauliX, PauliI, PauliX, PauliI, PauliX],
            [PauliI, PauliX, PauliX, PauliI, PauliI, PauliX, PauliX],
            [PauliI, PauliI, PauliI, PauliX, PauliX, PauliX, PauliX]
        ];
    // S4-6
    let zg = 
        [ 
            [PauliZ, PauliI, PauliZ, PauliI, PauliZ, PauliI, PauliZ],
            [PauliI, PauliZ, PauliZ, PauliI, PauliI, PauliZ, PauliZ],
            [PauliI, PauliI, PauliI, PauliZ, PauliZ, PauliZ, PauliZ] 
        ];
    // Measure the stabilizers
    let sx = SyndromeMeasOp(MeasureStabilizerGenerators(xg, _, MeasureWithScratch));
    let sz = SyndromeMeasOp(MeasureStabilizerGenerators(zg, _, MeasureWithScratch));
    let code = CSS(e, d, sx, sz);
    return code;
}

Next, we will discuss the error correction procedure.

## 3. A Theoretical Implementation of Quantum Error Correction

According to the theory, we can derive a theoretical implementation, whose circuit is shown below.

<div>
    <img src="img/theoretical.png" width="75%" style="width: 75%; margin: 15px auto 15px auto;"/>
</div>

$|q_{0-6}\rangle$ denotes the codeword to correct. $|a_{1-3}\rangle$ are 3 extra qubits (called *ancilla qubits*) used to measure the stabilizers $S_i$.

First, we measure $S_{1-3}$ via $|a_{1-3}\rangle$ for the syndrome to detect phase flip errors. The syndrome is then mapped ($\cal M$) to an error correction operation, which is applied to the codeword to correct errors ($\cal C$). Next, $|a_{1-3}\rangle$ are reused to measure $S_{4-6}$ to perform bit flip error correction.

### Example: Quantum Error Correction using Q# Library #

The theoretical implementation of quantum error correction with Steane Code is defined as `RecoverCSS` in Q# Library.

In [ ]:
// https://github.com/microsoft/quantumlibraries/blob/master/Standard/src/ErrorCorrection/Utils.qs
operation RecoverCSS (code : CSS, fnX : RecoveryFn, fnZ : RecoveryFn, logicalRegister : LogicalRegister) : Unit
{
    let (encode, decode, syndMeasX, syndMeasZ) = code!;
    let syndromeX = syndMeasX!(logicalRegister); // Syndrome computation: measure S1, S2, S3
    let recoveryOpX = fnX!(syndromeX);           // Mapping (M)
    Message($"X: {syndromeX} → {recoveryOpX}");
    ApplyPauli(recoveryOpX, logicalRegister!);   // Correcting phase flip errors (C)
    let syndromeZ = syndMeasZ!(logicalRegister); // Syndrome computation: measure S4, S5, S6
    let recoveryOpZ = fnZ!(syndromeZ);           // Mapping (M)
    Message($"Z: {syndromeZ} → {recoveryOpZ}");
    ApplyPauli(recoveryOpZ, logicalRegister!);   // Correcting bit flip errors (C)
}

We show its usage by introducing a bit flip error in the `EncodeAndDecode` example of the previous section.

In [3]:
open Microsoft.Quantum.Math; // for PI
open Microsoft.Quantum.ErrorCorrection; // import Error Correction Library

operation ErrorCorrection() : Unit {
    using ((data, auxQubits) = (Qubit(), Qubit[6])) {
        Rx(PI() / 3.0, data); // Constuct a non-trivial state to begin with
        
        let code = SteaneCode();
        let (encode, decode, syndMeasX, syndMeasZ) = code!;
        
        let codeword = encode!([data], auxQubits); // Encode data
        
        X(codeword![2]); // Introduce a bit flip error on the 3rd qubit
        
        let (fnX, fnZ) = SteaneCodeRecoveryFns(); // Mapping functions
        
        RecoverCSS(code, fnX, fnZ, codeword); // Correcting the bit flip error
        
        let (decodedData, decodedAux) = decode!(codeword); // Decode the codeword
        
        // Check the state
        Adjoint Rx(PI() / 3.0, data);
        Assert([PauliZ], [data], Zero, "State doesn't return to |0>");
    }
}

ErrorCorrection

In [4]:
%simulate ErrorCorrection

X: Syndrome([Zero,Zero,Zero]) → [PauliI,PauliI,PauliI,PauliI,PauliI,PauliI,PauliI]
Z: Syndrome([One,One,Zero]) → [PauliI,PauliI,PauliX,PauliI,PauliI,PauliI,PauliI]


()

We can see from the output that the bit flip error on the 3rd qubit is detected and corrected successfully.

## 4. The Problem of the Theoretical Implementation : Error Propagation

Ancilla qubits are the most vulnerable in the theoretical implementation. To explain this, we take $S_1$ measurement for an example. The following circuit presents the measurement detail.

<div>
    <img src="img/s1.png" width="25%" style="width: 25%; margin: 15px auto 15px auto;"/>
</div>

Since $S_1 = X \otimes I \otimes X \otimes I \otimes X \otimes I \otimes X$, the ancilla qubit are connected to 4 qubits $|q_{0,2,4,6}\rangle$ via CNOT.

Suppose a X error happens after H on the ancilla qubit $|a_1\rangle$, through CNOT, the X error will propagate to the controlled qubits. In this case, $|q_{0,2,4,6}\rangle$ each will get a bit flip.

Such propagation is fatal. Steane Code can only correct single qubit errors, the propagation causes errors on three qubits, which cannot be corrected by the correction procedure. In other words, without fault-tolerant consideration, theoretical implementation may introduce more errors than it can correct, getting things worse.

## 5. Fault-tolerant Error Correction

Before we elaborate the Steane Error Correction scheme, we must define fault-tolerant error correction to clarify the fault-tolerant requirements. The following definitions and the figures in the definitions are taken from [1].

**Definition 1 ($r$-Filter)** An $r$*-filter* projects any input onto states with errors of weight at most $r$.

In other words, $r$-filter defines a way to express a state which contains errors of weight at most $r$.

**Definition 2 (Ideal Decoder)** An *ideal decoder* performs an error-free procedure that takes a encoded state, corrects any errors and gives an unencoded state.

The graphical notations to represent these objects are as follows:

<div>
    <img src="img/notations.png" width="30%" style="width: 30%; margin: 15px auto 15px auto;"/>
</div>

Thick lines represent a single codeword (7 qubits in Steane Code), while the thin line of the output from the ideal decoder represents a single unencoded qubit.

**Definition 3 (Fault-tolerant Error Correction)** An error correction (EC) procedure is *fault-tolerant* if it satisfies the following two requirements:

<div style="float: left; clear: both;">
    <img src="img/eca.png" width="40%" style="width: 40%; margin: 15px auto 15px auto; float:left;"/>
</div>

<div style="float: left; clear: both;">
    <img src="img/ecb.png" width="48%" style="width: 48%; margin: 15px auto 15px auto; float:left;"/>
</div>

The number $s$ above the EC box denotes that the weight of errors occurring during the error correction procedure is no more than $s$.

EC A says that any codewords corrected by an error correction procedure with weight $s \le t$ errors happening during the procedure will have at most $s$ weight errors away from some encoded state. Note that this does not mean that the errors are handled correctly, only that the final state is near a codeword, which may be a wrong but a valid codeword.

EC B says that the codeword is corrected correctly if the total weight of errors in the input and during the correction is less than $t$. Note that the state of the output must be the correct one represented by the input.

### Concretization for Steane Code

We concretize the above requirements for Steane Code with $t = 1$. We assume all other parts of the whole circuit work properly, thus the codewords as the input to EC can contain no or single qubit error. Enumerating all the possible weight of errors in input and during correction, 

| ErrorInInput | ErrorDuringEC | Type |   |
|--------------|---------------|------|---|
| 0            | 0             | A    |   |
| 0            | 1             | A    |   |
| 1            | 0             | A    |   |
| 1            | 1             | A    | √ |
| 0            | 0             | B    | √ |
| 0            | 1             | B    | √ |
| 1            | 0             | B    | √ |

(when two configurations differ only in their types, since the requirement of type B is stronger than type A, we choose that of type B)

we get the following concrete requirements:

<div style="float: left; clear: both;">
    <img src="img/requirements.png" width="35%" style="width: 35%; margin: 15px auto 15px auto; float:left;"/>
</div>

EC 1 derives from EC A, saying that errors can be mitigated if their weight are not too much. EC 2, 3 and 4 derive from EC B, and they are the correctness guarantee that single errors can be corrected.

We then elaborate the Steane Error Correction scheme and then verify that the scheme satisfies the four requirements.

## 6. Steane Error Correction Scheme

The theoretical implementation exploits the property of Steane Code as a stabilizer code. It uses measurement of stabilizers to get syndromes and the type of errors. Instead, Steane Error Correction exploits more about the property of Steane Code as a CSS code, which uses a different way of syndrome computation, thereby mitigating the error propagation problem.

> Shor Error Correction scheme achieves fault-tolerance by modifying the theoretical implementation, readers can refer to [1], [4] for more details. Since *Shor error correction is rather laborious, requiring many extra gates and a low physical error rate to work well* [1], we choose to elaborate another scheme, Stean Error Correction, in this tutorial.

### 6.1 Steane Code as a CSS Code

A CSS Code $[[n, k_1 - k_2]]$ is constructed from two classical dual codes $C_1$ and $C_2$ where 1) $C_1$ and $C_2$ are $[n, k_1]$, $[n, k_2]$ classical linear codes, 2) $C_2 \subset C_1$, and 3) $C_1$ and $C_2^\bot$ each can correct errors of weight at most $t$. E.g. Steane Code is a $[[7, 1]]$ CSS Code constructed from $[7, 4]$ and $[7, 3]$ classical Hamming codes.

The codewords of a CSS code are of the form 

$$
\frac{1}{\sqrt{|C_2|}} \sum_{w \in C_2^\bot} |u + w\rangle
$$

where $u \in C_1$ reveals the logical codeword encoded the state. Thus, if we were to measure every qubits of the codeword, we will get the classical result $u + w$ for a random $w \in C_2^\bot$. In case of errors, the result will be $u + w + e$ where $e$ represents the type of errors. Since $u + w \in C_1$ is a classical codeword, we can apply the classical syndrome computation to detect errors by multiplying the parity check matrix and the result.

Steane Error Correction exploits this property that quantum errors can be transformed to classical errors for CSS codes.

### 6.2 Steane Error Correction Scheme

For gates between two qubits, we can define its *transversal* gate between two blocks of qubits. A transversal gate is constructed by applying the original gate between corresponding qubits of the two blocks. E.g. The transversal CNOT between two blocks of 3 qubit each, $|q_{0-2}\rangle$ to $|a_{0-2}\rangle$, is defined as:

<div>
    <img src="img/transversal.png" width="10%" style="width: 10%; margin: 15px auto 15px auto;"/>
</div>

where CNOT is performed between $\left(|q_0\rangle, |a_0\rangle\right)$, $\left(|q_1\rangle, |a_1\rangle\right)$ and $\left(|q_2\rangle, |a_2\rangle\right)$.

The circuit of Steane Error Correction is shown as follows.

<div>
    <img src="img/steane.png" width="60%" style="width: 60%; margin: 15px auto 15px auto;"/>
</div>

$|q_{0-6}\rangle$ and $|a_{0-6}\rangle$ each is a block of 7 qubits, and the CNOT in the circuit are transversal CNOT between the two blocks. H, Measure and C are applied to every qubit in the block.

Similar to the theoretical implementation, Steane Error Correction detects and corrects bit flip errors and phase flip errors separately. The left part of the circuit is used to deal with bit flip errors and the right part is for phase flip errors.

For bit flip errors, we first prepare a logical $|\overline{+}\rangle$ state for the ancilla block (A logical $|\overline{+}\rangle$ state is a $|+\rangle$ state encoded by Steane Code) and perform transversal CNOT from the codeword to the ancilla block. The transversal CNOT is used to propagate bit flip errors from the codeword to the ancilla qubits. Then we measure the ancilla qubits and treat the result as a classical codeword in $C_1$. We perform the classical syndrome computation $\cal{M}_c$, deduce the location of errors and then correct it on the codeword ($\cal{C}$).

For phase flip errors, we prepare a logical $|\overline{0}\rangle$ state and perform a similar correction procedure.

The reason why such scheme is fault-tolerant lies in the use of transversal CNOT gates. We explain from the view of error propagation.

First, except for the transversal CNOT gates, all other operations are single qubit operations, which means that errors taking place during the operation can only affect the qubit itself, but not other qubits. Thus, single qubit operations will not propagate errors.

Second, transversal CNOT gates are performed between corresponding qubits of two blocks, and more importantly, within each block, no qubit are used twice. Even errors are propagated to the other block via CNOT operations, only one qubit within each block will be affected. This is vital because we perform error correction at the granularity of block, and single qubit errors in a block can be corrected by Steane Code. In contrast, the ancilla qubits in the theoretical implementation are used in multiple CNOT operations, and thus errors can be spread to multiple qubits within one block using the common ancilla qubits as bridge.

Readers can refer to [1] and [2] fore more details.

### 6.3 Fault-tolerant State Preparation

Unlike the theoretical implementation where the initial state of ancilla qubits is $|0\rangle$, we need logical $|\overline{+}\rangle$ and $|\overline{0}\rangle$ to initialize the ancilla qubits in the Steane Error Correction scheme. As we see before, the encoding operation involves many two qubit operations so that a single error during encoding may cause many qubit errors in the final codeword. If we use wrong initial states for ancilla qubits, it may not only perform a wrong error correction but also destroy the information stored in the computational qubits.

Therefore, we need fault-tolerant state preparation for these ancilla qubits.

There are many ways for fault-tolerant state preparation, and Gottesman proposed a *trial-and-error* method which reuses the above error correction procedure [1].

1. We first prepare the target state (e.g. the logical $|\overline{+}\rangle$) using the encoding function.
2. Then we perform the above error correction procedure on the prepared state, but we use the procedure only to find if there is any error in the prepared state, not to correct it.
3. If we find any error, we simply discard the prepared state and repeat the above steps until no error is detected.

Note that, in step 2, since we reuse the same error correction procedure, we also need to prepare initial states for the ancilla qubits. But for this time, we simply use the encoding function to prepare these states and do no further verification. We explain the reason as follows.

Detecting with unverified ancilla qubits may get wrong results, but in order for a wrong prepared state to pass the detection, we need correlated errors both on the prepared states and the ancilla qubits, and these errors must take place in a *right* way that they succeed to hide themselves in the syndrome.

### 6.4 Implement using Q# #

We then detail how to implement the Steane Error Correction shceme using Q#. Readers can refer to `FTEC.qs` file for the complete codes.

The whole procedure is defined as below:

In [ ]:
operation FTSteaneEC(codeword : LogicalRegister) : Unit {
    let (encode, decode, syndMeasX, syndMeasZ) = (SteaneCode())!; // Only for encode
    
    // For bit flip errors
    using (ancillaQubits = Qubit[7]) {
        // Prepare initial state for ancilla qubits in a fault-tolerant way
        let ancilla = FTStatePreparation(BFE_StatePrepare, encode, ancillaQubits);
        // Syndrome computation
        let syndrome = BFE_Detect(codeword, ancilla);
        // Mapping and error correcting
        BFE_Correct(codeword, syndrome);
    }

    // For phase flip errors
    using (ancillaQubits = Qubit[7]) {
        let ancilla = FTStatePreparation(PFE_StatePrepare, encode, ancillaQubits);
        let syndrome = PFE_Detect(codeword, ancilla);
        PFE_Correct(codeword, syndrome);
    }
}

We will next detail `BFE_StatePrepare`, `BFE_Detect` and `BFE_Correct` in turn and leave `FTStatePreparation` to the end. Since the routines for phase flip errors are similar to those for bit flip errors, we omit them here for brevity.

`BFE_StatePrepare` prepares initial states in a non-fault-tolerant way, and it uses the `encode` operation from Q# library.

In [ ]:
operation BFE_StatePrepare(encode : EncodeOp, ancillaQubits : Qubit[]) : LogicalRegister {
    // For logical |+> state
    H(ancillaQubits[0]);
    return encode!([ancillaQubits[0]], ancillaQubits[1..6]);
}

`BFE_Detect` implements the fault-tolerant syndrome computation logic.

In [ ]:
operation BFE_Detect(codeword : LogicalRegister, ancilla : LogicalRegister) : Syndrome {
    // Transversal CNOT from codeword to ancilla qubits
    for (i in IndexRange(codeword!)) {
        CNOT(codeword![i], ancilla![i]);
    }

    // Measure ancilla qubits to get a classical codeword
    let results = ForEach(MResetZ, ancilla!);
    let ccodeword = ResultArrayAsInt(results);

    // Compute the syndrome by multiplying the parity check matrix with the classical codeword
    let parity = SteaneCodeParity();   // `SteaneCodeParity` returns the parity check matrix
    let syndrome = Mapped(ParityCheck(_, ccodeword), parity);  // Multiply each row of the matrix with the classical codeword

    return Syndrome(syndrome);
}

`BFE_Correct` corrects bit flip errors according to the syndrome, and it uses the `SteaneCodeRecoveryX` operation from Q# library for the mapping.

In [ ]:
operation BFE_Correct(codeword : LogicalRegister, syndrome : Syndrome) : Unit {
    // Mapping from the syndrome to error correcting operations (Mc)
    let recoverOp = SteaneCodeRecoveryZ(syndrome);
    // Correcting bit flip errors (C)
    ApplyPauli(recoverOp, codeword!);
}

We then detail the fault-tolerant state preparation `FTStatePreparation`. For generalization, `FTStatePreparation` isolates the states to prepare by accepting a `statePrepare` operation (e.g. `BFE_StatePrepare`) as input.

In [ ]:
operation FTStatePreparation(statePrepare : ((EncodeOp, Qubit[]) => LogicalRegister), encode : EncodeOp, ancillaQubits : Qubit[]) : LogicalRegister {
    mutable satisfied = true;
    repeat {
        // Step 1: We first prepare the target state using the given `statePrepare` operation
        let ancilla = statePrepare(encode, ancillaQubits);
        
        // Step 2: We detect if any errors in the prepared state.
        // The detection reuses the error correction procedure.
        using (auxQubits = Qubit[7]) {
            // Ancilla qubits used to assist detection are prepared in a non-fault-tolerant way.
            let aux = BFE_StatePrepare(encode, auxQubits);
            let syndrome = BFE_Detect(ancilla, aux);
            set satisfied = satisfied and isNoError(syndrome);
        }

        using (auxQubits = Qubit[7]) {
            let aux = PFE_StatePrepare(encode, auxQubits);
            let syndrome = PFE_Detect(ancilla, aux);
            set satisfied = satisfied and isNoError(syndrome);
        }
        
    } until (satisfied)
    fixup {
        // Step 3: If any error, we discard the prepared state and repeat.
        ResetAll(ancillaQubits);
        set satisfied = true;
    }
    return LogicalRegister(ancillaQubits);
}

We prepare all the above codes in the workspace, readers can verify its correctness by correcting the same bit flip error as we did in the previous example.

In [1]:
%workspace

FTEC._DetectCSS, FTEC._RecoverCSS, FTEC.ApplyError, FTEC.BFE_Correct, FTEC.BFE_Detect, FTEC.BFE_StatePrepare, FTEC.BinaryArrayAsInt, FTEC.FTStatePreparation, FTEC.FTSteaneEC, FTEC.FTSteaneEC_ForVerification, FTEC.isNoError, FTEC.ParityCheck, FTEC.PFE_Correct, FTEC.PFE_Detect, FTEC.PFE_StatePrepare, FTEC.SteaneCodeParity, FTEC.TheoreticalSteaneEC, FTEC.Verify, FTEC.VerifyEC

In [2]:
open Microsoft.Quantum.Math;
open Microsoft.Quantum.ErrorCorrection;
operation FTErrorCorrection() : Unit {
    using ((data, auxQubits) = (Qubit(), Qubit[6])) {
        // Prepare and encode
        Rx(PI() / 3.0, data);
        
        let code = SteaneCode();
        let (encode, decode, syndMeasX, syndMeasZ) = code!;
        let codeword = encode!([data], auxQubits);
        
        // Introduce the same bit flip error on the 3rd qubit
        X(codeword![2]);
        
        // !!! Perform fault-tolerant Steane Error Correction
        FTEC.FTSteaneEC(codeword);
        
        // Decode and check
        let (decodedData, decodedAux) = decode!(codeword);
        
        Adjoint Rx(PI() / 3.0, data);
        Assert([PauliZ], [data], Zero, "State doesn't return to |0>");
    }
}

FTErrorCorrection

In [3]:
%simulate FTErrorCorrection

()

## 7. Verification

Now, let us verify that the Steane Error Correction scheme satisfies all the fault-tolerant requirements, i.e, **EC 1, 2, 3, 4**.

To perform verification, we need to introduce *errors of weight at most 1* during error correction, which requires us to design a faulty simulator.

### 7.1 Faulty Simulator

There are two requirements for our faulty simulator:

1. Errors of weight at most 1 mean that errors can only occur during single qubit operations. Two-qubit operations (e.g. CNOT) are not allowed to have errors because they will propagate the error and amplify 1 weight errors to 2 weight errors.
2. Since the total weight of errors during error correction is no more than 1, and an error has weight at least 1, we can only introduce at most one error in the whole procedure.

Then, we will present three key points in the implementation, readers can refer to `FaultySimulator.cs` for the complete codes.

**1. Faulty single qubit gates are defined by inheriting and overriding corresponding gate implementations**

The single gates used in the error correction procedure is H, X and Z. We create a new faulty gate for each of them by inheriting the original implementation. We introduce errors by overriding the original single qubit operation.

For example, we define a new `H` class inherited from the error-free implementation `QSimH` and override the `Body` function.

In [ ]:
// C#

// `QSimH` is the original error-free implementation of H gate
public class H : QSimH {
    private FaultySimulator sim;
    public H(FaultySimulator m) : base(m) { sim = m; }

    // Single qubit operation of H
    public override Func<Qubit, QVoid> Body {
        get {
            // Get the original operation
            Func<Qubit, QVoid> original = base.Body;

            return (qubit => {
                // Apply error
                if (sim.isErrorHappened()) sim.applyError(qubit);
                // and perform the original operation
                return original(qubit);
            });
        }
    }
} // class H

**2. Errors are defined as Pauli errors**

We randomly choose to apply X or Z Pauli errors.

In [ ]:
// Apply a Pauli error to `qubit`
void applyError(Qubit qubit) {
    double p = rnd.NextDouble();
    if (p < 0.5) {
        // X error
        QSimX x = new QSimX(this);
        x.Body(qubit);
    } else {
        // Z error
        QSimZ z = new QSimZ(this);
        z.Body(qubit);
    }
}

**3. Control messages are used to to enable/disable the faulty mechanism**

We use `Message` with certain content `FaultyEnabled` and `FaultyDisabled` to enable and disable the faulty mechanism.

All operations after `FaultyEnabled` may suffer from at most one single qubit error (according to the above two requirements). And all operations after `FaultyDisabled` is error-free.

The usage is shown as below:

In [ ]:
operation AnOperation() : Unit {
    Message("FaultyEnabled");
    // Operations here may suffer from a single qubit error
    Message("FaultyDisabled");
    // Operations here are error-free
}

Now, we design a verification framework using the faulty simulator.

### 7.2 Verification Framework

The verification framework is based on the single qubit error correction example in previous sections except that we introduce three control points `errorInInput`, `errorDuringEC` and `isTypeA` according to the requirement table shown as below.

| Requirement | ErrorInInput | ErrorDuringEC | Type |
|-------------|--------------|---------------|------|
| EC 1        | 1            | 1             | A    |
| EC 2        | 0            | 0             | B    |
| EC 3        | 1            | 0             | B    |
| EC 4        | 0            | 1             | B    |

The verification procedure is:

1. Create a input codeword by encoding a single qubit with a non-trivial state using Steane Code.
2. Apply an error to the input codeword if there should be one (Control point 1: `errorInInput`).
3. Perform the fault-tolerant error correction with/without the faulty mechanism (Control point 2: `errorDuringEC`).
4. Check the final state according to the type of the requirement (Control point 3: `isTypeA`).
5. Repeat 100 times and count how many times the verification fails.

The detail is shown as below:

In [ ]:
operation Verify(errorInInput : Bool, errorDuringEC : Bool, isTypeA : Bool) : Unit {
    // Perform verification 100 times and count how many times the verification fails.
    let N = 100;
    mutable nFail = 0;

    for (i in 0..N-1) {
        using ((data, auxQubits) = (Qubit(), Qubit[6])) {
            // Construct a non-trivial state as input
            Rx(PI() / 3.0, data);

            let code = SteaneCode();
            let (encode, decode, syndMeasX, syndMeasZ) = code!;
            let codeword = encode!([data], auxQubits);
            
            // Control point 1
            if (errorInInput) { ApplyError(codeword![1]); }
            
            // Control point 2: Enable the faulty mechanism according to errorDuringEC.
            if (errorDuringEC) { Message("FaultyEnabled"); }
            FTSteaneEC_ForVarifaction(codeword, errorDuringEC);
            if (errorDuringEC) { Message("FaultyDisabled"); }
            
            // Control point 3
            mutable noError = true;
            if (isTypeA) {
                // According to Type A, the final codeword is near a valid codeword 
                // but it is not necessarily the original one.
                // Thus, we only check the validity of the codeword.
                
                // 1-filter can be implemented as a detection after correcting one single qubit error.
                TheoreticalSteaneEC(codeword);
                set noError = _DetectCSS(code, codeword);
            } else {
                // According to Type B, the final state after an ideal decoder should be the original one
                
                // Ideal decoder
                TheoreticalSteaneEC(codeword);
                let (decodedData, decodedAux) = decode!(codeword);
                
                // Check if it is the original state
                Adjoint Rx(PI() / 3.0, data);
                set noError = (M(data) == Zero);
            }

            if (not noError) {
                set nFail = nFail + 1;
            }

            ResetAll([data] + auxQubits);
        }

        if (i != 0 and i % 10 == 0) {
            Message($"[{i} / {N}] Fail Ratio: {IntAsDouble(nFail) / IntAsDouble(i)}");
        }
    }
    Message($"Total Fail Ratio: {IntAsDouble(nFail) / IntAsDouble(N)}");
}

There is one more thing to clarify. In the original paper, they prove that the Steane Error Correction scheme is fault-tolerant under the assumption that the initial states of ancilla qubits are prepared without any error. The verification may fail if there are errors containing in the initial states, thus we verify with `FTSteaneEC_ForVarifaction` which replaces the fault-tolerant state preparation with perfect state preparation.

Readers can run the complete verification under `verification` folder by

```
dotnet run
```

We show the output as follow:

In [ ]:
[Verify EC1] errorInInput : true, errorDuringEC : true, isTypeA : true
[10 / 100] Fail Ratio: 0
[20 / 100] Fail Ratio: 0
[30 / 100] Fail Ratio: 0
[40 / 100] Fail Ratio: 0
[50 / 100] Fail Ratio: 0
[60 / 100] Fail Ratio: 0
[70 / 100] Fail Ratio: 0
[80 / 100] Fail Ratio: 0
[90 / 100] Fail Ratio: 0
Total Fail Ratio: 0
[Verify EC2] errorInInput : false, errorDuringEC : false, isTypeA : false
[10 / 100] Fail Ratio: 0
[20 / 100] Fail Ratio: 0
[30 / 100] Fail Ratio: 0
[40 / 100] Fail Ratio: 0
[50 / 100] Fail Ratio: 0
[60 / 100] Fail Ratio: 0
[70 / 100] Fail Ratio: 0
[80 / 100] Fail Ratio: 0
[90 / 100] Fail Ratio: 0
Total Fail Ratio: 0
[Verify EC3] errorInInput : false, errorDuringEC : true, isTypeA : false
[10 / 100] Fail Ratio: 0
[20 / 100] Fail Ratio: 0
[30 / 100] Fail Ratio: 0
[40 / 100] Fail Ratio: 0
[50 / 100] Fail Ratio: 0
[60 / 100] Fail Ratio: 0
[70 / 100] Fail Ratio: 0
[80 / 100] Fail Ratio: 0
[90 / 100] Fail Ratio: 0
Total Fail Ratio: 0
[Verify EC4] errorInInput : true, errorDuringEC : false, isTypeA : false
[10 / 100] Fail Ratio: 0
[20 / 100] Fail Ratio: 0
[30 / 100] Fail Ratio: 0
[40 / 100] Fail Ratio: 0
[50 / 100] Fail Ratio: 0
[60 / 100] Fail Ratio: 0
[70 / 100] Fail Ratio: 0
[80 / 100] Fail Ratio: 0
[90 / 100] Fail Ratio: 0
Total Fail Ratio: 0

We can see that Steane Error Correction scheme passes all the verification.

## 8. Further Exploration

### 8.1 A More Realistic Faulty Simulator

We have prepared a more realistic faulty simulator by relaxing the constraints in the verification.

First, we allow errors in two-qubit operations. Second, errors can occur in a local and Markov manner [3] rather than only once.

We compare the Steane Error Correction Scheme and the theoretical implementation using the new simulator. Readers can find the codes in `exploration` folder.

The following shows the result of one run. We can see that `FTSteaneEC` has a lower fail ratio than `TheoreticalSteaneEC`.

In [ ]:
TheoreticalSteaneEC
[10 / 100] Fail Ratio: 0.2
[20 / 100] Fail Ratio: 0.2
[30 / 100] Fail Ratio: 0.16666666666666666
[40 / 100] Fail Ratio: 0.175
[50 / 100] Fail Ratio: 0.22
[60 / 100] Fail Ratio: 0.2
[70 / 100] Fail Ratio: 0.2
[80 / 100] Fail Ratio: 0.2
[90 / 100] Fail Ratio: 0.2111111111111111
Total Fail Ratio: 0.22
FTSteaneEC
[10 / 100] Fail Ratio: 0
[20 / 100] Fail Ratio: 0.05
[30 / 100] Fail Ratio: 0.03333333333333333
[40 / 100] Fail Ratio: 0.025
[50 / 100] Fail Ratio: 0.04
[60 / 100] Fail Ratio: 0.05
[70 / 100] Fail Ratio: 0.05714285714285714
[80 / 100] Fail Ratio: 0.0625
[90 / 100] Fail Ratio: 0.05555555555555555
Total Fail Ratio: 0.06

### 8.2 References

The following materials are referred to when writing the tutorial:

- [1] Daniel Gottesman. An introduction to quantum error correction and fault-tolerant quantum computation.
- [2] Andrew Steane. Active stabilisation, quantum computation and quantum state synthesis.
- [3] Eleanor Rieffel and Wolfgang Polak. Quantum Computing - A Gentle Introduction.
- [4] Michael Nielsen and Isaac Chuang. Quantum Computation and Quantum Information.